<a href="https://colab.research.google.com/github/kradkahaddi/LANL-Earthquake-challenge-on-Kaggle/blob/master/LANL_Challenge_LGBM_and_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing google file interface
from google.colab import files

#creating a variable to store kaggle authenticator .json file
uploaded = files.upload()

#shifting the kaggle variable to the appropriate folder
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json 

In [0]:
#list the datasets available
!kaggle competitions list

In [0]:
#WARNING DONT RUN UNNECESSARILY
#delete old instance of data
!find . -type f -name '*.zip' -delete

In [0]:
#download the appropriate dataset - in this case, the LANL-Earthquake-Prediotion dataset
!kaggle competitions download -c LANL-Earthquake-Prediction

  0% 0.00/33.3k [00:00<?, ?B/s]
100% 33.3k/33.3k [00:00<00:00, 28.1MB/s]
 93% 225M/242M [00:01<00:00, 149MB/s]
100% 242M/242M [00:01<00:00, 132MB/s]
100% 2.02G/2.03G [00:35<00:00, 22.1MB/s]
100% 2.03G/2.03G [00:36<00:00, 60.4MB/s]


In [0]:
!ls

In [0]:
#create folders for test data and unzip files
!mkdir test
!unzip test.zip -d test
!unzip train.csv.zip

                        

---


                        **RERUN ONLY AFTER THIS CELL IN CASE OF CRASHES**

---


In [0]:
#importing libraries and declaring constants

import os
import gc
import numpy as np
from numpy.fft import *
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pywt 
from statsmodels.robust import mad
import scipy
from scipy import signal
from scipy.signal import butter, deconvolve
import scipy.stats 
import glob
import warnings
warnings.filterwarnings('ignore')

#IMPORTING THE MODEL
import lightgbm as lgb
from xgboost import XGBRegressor


SAMPLE_RATE = 4000000
SIGNAL_LEN  = 150000      

print("import and globals setup complete")

import and globals setup complete


In [0]:
#DETECT PEAKS LIBRARY FROM MARCOS DUARTE
# https://nbviewer.jupyter.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
# %load ./../functions/detect_peaks.py
"""Detect peaks in data based on their amplitude and other features."""

from __future__ import division, print_function
import numpy as np

__author__ = "Marcos Duarte, https://github.com/demotu/BMC"
__version__ = "1.0.5"
__license__ = "MIT"


def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    """Detect peaks in data based on their amplitude and other features.

    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height (if parameter
        `valley` is False) or peaks that are smaller than maximum peak height
         (if parameter `valley` is True).
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).

    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.

    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`
    
    The function can handle NaN's 

    See this IPython Notebook [1]_.

    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb

    Examples
    --------
    >>> from detect_peaks import detect_peaks
    >>> x = np.random.randn(100)
    >>> x[60:81] = np.nan
    >>> # detect all peaks and plot data
    >>> ind = detect_peaks(x, show=True)
    >>> print(ind)

    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # set minimum peak height = 0 and minimum peak distance = 20
    >>> detect_peaks(x, mph=0, mpd=20, show=True)

    >>> x = [0, 1, 0, 2, 0, 3, 0, 2, 0, 1, 0]
    >>> # set minimum peak distance = 2
    >>> detect_peaks(x, mpd=2, show=True)

    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # detection of valleys instead of peaks
    >>> detect_peaks(x, mph=-1.2, mpd=20, valley=True, show=True)

    >>> x = [0, 1, 1, 0, 1, 1, 0]
    >>> # detect both edges
    >>> detect_peaks(x, edge='both', show=True)

    >>> x = [-2, 1, -2, 2, 1, 1, 3, 0]
    >>> # set threshold = 2
    >>> detect_peaks(x, threshold = 2, show=True)

    Version history
    ---------------
    '1.0.5':
        The sign of `mph` is inverted if parameter `valley` is True
    
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
        if mph is not None:
            mph = -mph
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
            if mph is not None:
                mph = -mph
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


def _plot(x, mph, mpd, threshold, edge, valley, ax, ind):
    """Plot results of the detect_peaks function, see its help."""
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        if ax is None:
            _, ax = plt.subplots(1, 1, figsize=(8, 4))

        ax.plot(x, 'b', lw=1)
        if ind.size:
            label = 'valley' if valley else 'peak'
            label = label + 's' if ind.size > 1 else label
            ax.plot(ind, x[ind], '+', mfc=None, mec='r', mew=2, ms=8,
                    label='%d %s' % (ind.size, label))
            ax.legend(loc='best', framealpha=.5, numpoints=1)
        ax.set_xlim(-.02*x.size, x.size*1.02-1)
        ymin, ymax = x[np.isfinite(x)].min(), x[np.isfinite(x)].max()
        yrange = ymax - ymin if ymax > ymin else 1
        ax.set_ylim(ymin - 0.1*yrange, ymax + 0.1*yrange)
        ax.set_xlabel('Data #', fontsize=14)
        ax.set_ylabel('Amplitude', fontsize=14)
        mode = 'Valley detection' if valley else 'Peak detection'
        ax.set_title("%s (mph=%s, mpd=%d, threshold=%s, edge='%s')"
                     % (mode, str(mph), mpd, str(threshold), edge))
        # plt.grid()
        plt.show()

In [0]:
"""
I am really grateful the work put into this public kernel that explains denoising:

https://www.kaggle.com/tarunpaparaju/lanl-earthquake-prediction-signal-denoising

The functions are borrowed directly from there. 
"""

def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)
  
  
def high_pass_filter(x, low_cutoff=1000, SAMPLE_RATE=SAMPLE_RATE):
    """
    From @randxie https://github.com/randxie/Kaggle-VSB-Baseline/blob/master/src/utils/util_signal.py
    Modified to work with scipy version 1.1.0 which does not have the fs parameter
    """
    
    # nyquist frequency is half the sample rate https://en.wikipedia.org/wiki/Nyquist_frequency
    nyquist = 0.5 * SAMPLE_RATE
    norm_low_cutoff = low_cutoff / nyquist
    
    # Fault pattern usually exists in high frequency band. According to literature, the pattern is visible above 10^4 Hz.
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig


def denoise_signal(x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per")
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest(coeff[-level])

    # Calculate the univeral threshold
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')

In [0]:
#Finds the max rate change of order of difference between peaks and the surrounds - similar to an tangent

def left_side_diff_max(X, mph = 0, mpd =0):
  indices = []
  indices = detect_peaks(X, mph = mpd, mpd = mpd)
  diff = list()
  
  surrounding_range = mpd//2
  
  for index in indices:
    range_fn = index - surrounding_range -1
    if (range_fn >=0):
      diff.append((X[index] - X[range_fn: index -1].mean())/surrounding_range)
    else:
      diff.append((X[index] - X[0:index -1].mean())/(index-1))
    
  diff_pd = pd.Series(diff)
  
  return diff_pd.max()
   
    

In [0]:
#Finds the avg rate change of order of difference between peaks and the surrounds - similar to an tangent

def left_side_diff_mean(X, mph = 0, mpd =0):
  indices = []
  indices = detect_peaks(X, mph = mph, mpd = mpd)
  diff = list()
  
  surrounding_range = mpd//2
  
  for index in indices:
    range_fn = index - surrounding_range -1
    if (range_fn >=0):
      diff.append((X[index] - X[range_fn: index -1].mean())/surrounding_range)
    else:
      diff.append((X[index] - X[0:index -1].mean())/(index-1))
  
  diff_pd = pd.Series(diff)
  
  return diff_pd.mean()


In [0]:
#Finds the max rate change of order of difference between peaks and the surrounds - similar to an tangent

def right_side_diff_max(X, mph = 0, mpd =0):
  indices = []
  indices = detect_peaks(X, mph = mph, mpd = mpd)
  diff = list()
  
  surrounding_range = mpd//2
  
  for index in indices:
    range_fn = index + surrounding_range + 1
    if (range_fn <=len(X)):
      diff.append((X[index] - X[index+1: range_fn].mean())/surrounding_range)
    else:
      diff.append((X[index] - X[index+1: len(X)].mean())/(len(X) - index - 1))
    
  diff_pd = pd.Series(diff)
  
  return diff_pd.max()
   
    

In [0]:
#Finds the max rate change of order of difference between peaks and the surrounds - similar to an tangent

def right_side_diff_mean(X, mph = 0, mpd =0):
  indices = []
  indices = detect_peaks(X, mph = mph, mpd = mpd)
  diff = list()
  
  surrounding_range = mpd//2
  
  for index in indices:
    range_fn = index + surrounding_range + 1
    if (range_fn <=len(X)):
      diff.append((X[index] - X[index+1: range_fn].mean())/surrounding_range)
    else:
      diff.append((X[index] - X[index+1: len(X)].mean())/(len(X) - index - 1))
  
  diff_pd = pd.Series(diff)
  
  return np.mean(diff_pd)
   
    

In [0]:
def num_groups_data (X, mph =0):
  
  val = list(np.abs(X))
  count = 1
  
#   indices = list()
#   indices_sorted = list()
  
  indices = np.array(detect_peaks(val,mph = mph))
#   indices_sorted.append(list(indices).sort()) #redundant but for the sake of comprehensiveness... c'est la vie!
  
  maxi = 501
  for iterate in range(len(indices) -1):
    diff = indices[iterate +1] - indices[iterate]
    
    if ( maxi <diff):
      maxi = diff
    
  if (maxi > 2000):
    maxi=2000 #500 is too little or diff was > 1000
  
  for iterate in range(len(indices)- 1):
    diff = indices[iterate +1] - indices[iterate]
    if (diff>maxi):
      count +=1
  
  return count
  
    
    

In [0]:
def spread_of_data (X, mph=0):
  X_list = list(np.abs(X))
  
  
  indices = np.array(detect_peaks(X_list,mph = mph))
#   indices_sorted.append(list(indices).sort()) #redundant but for the sake of comprehensiveness... c'est la vie!

  if len(indices) > 0:
    return indices[len(indices) - 1] - indices[0] 
  

In [0]:
eg = pd.Series([1,2,3,4,5,6,7])
eg[7-3:6]

                           

---


                           **VISUAL DATA ANALYSIS**
                           

---



In [0]:
gc.collect()

188

In [0]:
#Test: print plots in a loop
%%time

iteration = 0

train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

for df in train:
    iteration +=1
    
    for i in range(20):
      continue
    
    denoised_acoustic_data = high_pass_filter(denoise_signal(df.acoustic_data, wavelet='haar', level=1), low_cutoff = 10000, SAMPLE_RATE = 4000000)
    acoustic_data = list(df.acoustic_data)
    time_to_failure = list(df.time_to_failure)
    
    fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(16,8))
    
    ax1.plot(acoustic_data, 'crimson')
    ax1.set_title("original acoustic data", fontsize = 16)
    
    ax2.plot(denoised_acoustic_data, 'mediumvioletred')
    ax2.set_title("denoised data", fontsize = 16)
    
    ax3 = ax1.twinx()
    ax3.plot(time_to_failure, 'darkmagenta')
    ax3.set_title("time to failure", fontsize = 16)
    
    ax4 = ax2.twinx()
    ax4.plot(time_to_failure, 'darkmagenta')
    ax4.set_title("time to failure", fontsize = 16)
    
    x = input("Continue? y/n ")
    if(x=='n'):
      print("iteration is ", iteration)
      break
    
    



---

**PREDICTION OF EARTHQUAKES**

In [0]:
#generating features
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(scipy.stats.kurtosis(X))
    strain.append(scipy.stats.skew(X))
    strain.append(np.quantile(X,0.01))
    strain.append(np.quantile(X,0.05))
    strain.append(np.quantile(X,0.95))
    strain.append(np.quantile(X,0.99))
    strain.append(np.abs(X).max())
    strain.append(np.abs(X).mean())
    strain.append(np.abs(X).std())
    
    
#     strain.append(len(detect_peaks(X, mph=20))) - discard
#     strain.append(len(detect_peaks(X, mph=50)))
#     strain.append(len(detect_peaks(X, mph=100)))
    
    #measure of sustained activity
    strain.append(len(detect_peaks(np.abs(X), mph=20, mpd=3000))) 
    strain.append(len(detect_peaks(np.abs(X), mph=50, mpd=1000)))
    strain.append(len(detect_peaks(np.abs(X), mph=100, mpd=20)))
    
    #measuring rate of change    - keep
    strain.append(right_side_diff_mean(X, mph=50, mpd =500))
    strain.append(right_side_diff_max(X, mph=50, mpd =500))
    strain.append(left_side_diff_mean(X, mph=50, mpd =500))
    strain.append(left_side_diff_max(X, mph=50, mpd =500))
    
#     strain.append(spread_of_data (X, mph =50)) -discard
    strain.append(spread_of_data (X, mph =100))

#measuring rate of change around tall peaks - discard
#     strain.append(right_side_diff_mean(X, mph=100, mpd =50))
#     strain.append(right_side_diff_max(X, mph=100, mpd =1000))
#     strain.append(left_side_diff_mean(X, mph=100, mpd =50))
#     strain.append(left_side_diff_max(X, mph=100, mpd =1000))

    #measuring grouping of data
    strain.append(num_groups_data(X, mph =50))
    strain.append(num_groups_data(X, mph =100))
    
    
    return pd.Series(strain)

In [0]:
gc.collect()

12

In [0]:
%%time
train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

CPU times: user 2.21 ms, sys: 3.62 ms, total: 5.83 ms
Wall time: 10.2 ms


In [0]:
%%time
iterate = 0
X_train = pd.DataFrame()
y_train = pd.Series()
X_valid = pd.DataFrame()
y_valid = pd.Series()


train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

for df in train:
    denoised_acoustic_data = high_pass_filter(denoise_signal(df.acoustic_data, wavelet='haar', level=1), low_cutoff = 10000, SAMPLE_RATE = 4000000) 
    ch = gen_features(denoised_acoustic_data)
#     ch = gen_features(df.acoustic_data)
    
    if (iterate < 3500):
      X_train = X_train.append(ch, ignore_index=True)
      y_train = y_train.append(pd.Series(df['time_to_failure'].values[-1]))
    else:
      X_valid = X_valid.append(ch, ignore_index=True)
      y_valid = y_valid.append(pd.Series(df['time_to_failure'].values[-1]))
    
    iterate +=1
    
X_train = X_train.fillna(0.0)
X_valid = X_valid.fillna(0.0)

CPU times: user 10min 54s, sys: 49.7 s, total: 11min 44s
Wall time: 11min 52s


In [0]:
X_train = X_train.fillna(0.0)
X_valid = X_valid.fillna(0.0)

In [0]:
X_train.describe()

In [0]:
X_valid.describe()

In [0]:
#XGB hyperparameter sets are (n=4k, rate=0.025) and (n=10k, rate=0.01)
my_model = XGBRegressor(n_estimators=1000, learning_rate = 0.01, random_state=0)

my_model.fit(X_train,y_train,eval_metric='mae')

[17:16:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
#WITHOUT TEST VALID SPLIT

#Hyper parameters  (2000, 0.025) = 1.5558
#                  (1500, 0.05)  = 1.3577
#                  (2000, 0.05)  = 1.2070
#                  (3000, 0.025) = 1.3712
#                  (3000, 0.05)  = 0.9592
#                  (3500, 0.025) = 1.2864
#                  (4000, 0.025) = 1.2130
#                 (10000, 0.001) = 1.9647
#                 (10000, 0.01)  = 1.2085
#                 (20000, 0.001) = 1.8432
#                 (20000, 0.01)  = 0.7864
#                 (20000, 0.005) = 1.2162
# choosing (10000, 0.01) and (4000, 0.025) as hyperparameters

# without basic peak info - 1.8804
# without basic peak info and abs stats - 1.8824
# with abs stats removed - 1.8799

# with 3600 train-set and basic peak removed - 1.9236
# with (10000, 0.01) and b.p. removed - 1.1243
# with (5000, 0.01) and b.p. removed - 1.4870
# with (500, 0.01) and b.p. removed - 1.9994


# with (500, 0.05) and b.p. removed - 1.7232 - overfit

from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_train)

MAE = mean_absolute_error(predictions,y_train)
print(MAE)


1.8412690642361813


In [0]:
#WITH TEST VALID SPLIT
#with basic peak count - 2.3011
#with basic peak count removed - 2.2908
#with basic peak count and abs stats removed -2.2929
#with abs stats removed - 2.3008
#with 3600 train-set and basic peak removed - 2.1932
# with (10000, 0.01) and b.p. removed - 2.3186
# with (5000, 0.01) and b.p. removed - 2.2508
# with (500, 0.01) and b.p. removed - 2.1887



# with (500, 0.05) and b.p. removed - 2.2362 -overfit

from sklearn.metrics import mean_absolute_error

val_predictions = my_model.predict(X_valid)

val_MAE = mean_absolute_error(val_predictions,y_valid)
print(val_MAE)


2.5443048792521563


In [0]:
%%time
X_test = pd.DataFrame()

for file_name in glob.glob('test/*.csv'):
    
  data = pd.read_csv(file_name)
  denoised_data = high_pass_filter(denoise_signal(data.acoustic_data, wavelet='haar', level=1), low_cutoff = 10000, SAMPLE_RATE = 4000000)
  
  ft = gen_features(denoised_data)
  X_test = X_test.append(ft, ignore_index=True)
  X_test = X_test.fillna(0.0)
  

CPU times: user 5min 38s, sys: 16.6 s, total: 5min 54s
Wall time: 6min 2s


In [0]:
#CREATING AN OUTPUT VARIABLE
test_predictions = pd.DataFrame({'seg_id':[],'time_to_failure':[]})
# test_predictions.columns = ['seg_id','time_to_failure']
test_predictions.columns

for file_name in os.listdir('test/'):
  seg_name = file_name
  name_seg = seg_name.replace('.csv','')

  test_predictions = test_predictions.append({'seg_id':name_seg}, ignore_index= True)

In [0]:
X_test.describe()

In [0]:
test_predictions.shape

In [0]:
my_predictions = my_model.predict(X_test)
test_predictions.time_to_failure = my_predictions
test_predictions.to_csv('output8_XGB.csv')

In [0]:
test_predictions.describe()



---


                                               ***LIGHT-GBM ALGORITHM ATTEMPT***

---



In [0]:
from lightgbm import LGBMRegressor
my_lgbm_model = LGBMRegressor(max_depth = -1, n_estimators =400, learning_rate =0.01, random_state =0)
my_lgbm_model.fit(X_train, y_train, eval_metric ='mae')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
#LGBM Model hyper parameter tuning
#                  (2000, 0.025) = 
#                  (1500, 0.05)  = 
#                  (2000, 0.05)  = 
#                  (3000, 0.025) = 
#                  (3000, 0.05)  = 
#                  (3500, 0.025) = 
#                  (4000, 0.025) = 
#                 (10000, 0.001) = 1.5212 
#                 (10000, 0.01)  = 0.2646
#                 (10000, 0.005) = 0.6516
#                 (20000, 0.001) = 
#                 (20000, 0.01)  = 
#                 (20000, 0.005) = 

# with abs stats and basic peak count removed - 1.3851
# with abs stats removed - 1.3697
# with 3600 train-set and basic peak removed - 1.4387
# with (10000, 0.01) and b.p. removed - 0.2399
# with (5000, 0.01) and b.p. removed - 0.5780
# with (500, 0.01) and b.p. removed - 1.6743
# with (400, 0.01) and b.p. removed - 1.7414
# with (100, 0.01) and b.p. removed - 2.1678 -underfit

# with (300, 0.01) and b.p. removed - 1.8233
# with (250, 0.01) and b.p. removed - 1.8741

# with (500, 0.05) and b.p. removed - 0.9801 -overfit

from sklearn.metrics import mean_absolute_error

predictions_lgbm = my_lgbm_model.predict(X_train)

MAE_lgbm = mean_absolute_error(predictions_lgbm,y_train)
print(MAE_lgbm)


1.8536927301967585


In [0]:
#LGBM VALIDATION
#WITH TEST VALID SPLIT
# with basic peak count - 2.3351
# with basic peak count removed - 2.316
# with basic peak count and abs stats removed - 2.3163
# with abs stats removed - 2.3261
# with 3600 train-set and basic peak removed - 2.2400
# with (10000, 0.01) and b.p. removed - 2.3665
# with (5000, 0.01) and b.p. removed - 2.3340
# with (500, 0.01) and b.p. removed - 2.2053
# with (400, 0.01) and b.p. removed - 2.1895
# with (100, 0.01) and b.p. removed - 2.2007 - underfit

# with (300, 0.01) and b.p. removed - 2.1769
# with (250, 0.01) and b.p. removed - 2.1615

# with (500, 0.05) and b.p. removed - 2.3007 - overfit

from sklearn.metrics import mean_absolute_error

val_predictions_lgbm = my_lgbm_model.predict(X_valid)

val_MAE_lgbm = mean_absolute_error(val_predictions_lgbm,y_valid)
print(val_MAE_lgbm)


2.5658551834032295


In [0]:
!ls

In [0]:
#CREATING AN OUTPUT VARIABLE
lgbm_test_predictions = pd.DataFrame({'seg_id':[],'time_to_failure':[]})
# test_predictions.columns = ['seg_id','time_to_failure']
lgbm_test_predictions.columns

for file_name in os.listdir('test/'):
  seg_name = file_name
  name_seg = seg_name.replace('.csv','')

  lgbm_test_predictions = lgbm_test_predictions.append({'seg_id':name_seg}, ignore_index= True)

In [0]:
X_test.describe()

In [0]:
lgbm_test_predictions.describe()

In [0]:
my_lgbm_predictions = my_lgbm_model.predict(X_test)
lgbm_test_predictions.time_to_failure = my_lgbm_predictions
lgbm_test_predictions.to_csv('LGBM_output11.csv')

In [0]:
lgbm_test_predictions.describe()



---


**CATBOOST ALGOL TRY**

---



In [0]:
!pip install catboost

In [0]:
#CatBoost Model
from catboost import CatBoostRegressor, Pool

train_pool = Pool(X_train, y_train)
m = CatBoostRegressor(iterations=10000, loss_function='MAE', boosting_type='Ordered')
m.fit(X_train, y_train, silent=True)
m.best_score_



---


**RESIDUAL CODE AFTER THIS POINT**

---



In [0]:
%%time
denoised_acoustic_data = high_pass_filter(denoise_signal(signals, wavelet='haar', level=1), low_cutoff = 10000, SAMPLE_RATE = 4000000)

In [0]:
file_names[1]

In [0]:
import glob
i = 0
for f in glob.glob('test/*csv'):
  print(f)
  break


In [0]:
import glob 

file_names = [f for f in glob.glob('test/*.csv')]
len(file_names)


In [0]:
sample_output = pd.read_csv('sample_submission.csv','r')
sample_output.columns

In [0]:
for file_name in glob.glob('test/*.csv'):
  seg_name = file_name
  name_seg = seg_name.replace('.csv','')

  test_predictions = test_predictions.append({'seg_id':name_seg}, ignore_index= True)

In [0]:
seg_name = file_name
  name_seg = seg_name.replace('.csv','')
  nameseg = name_seg.replace('test/','')

  test_predictions = test_predictions.append({'seg_id':nameseg}, ignore_index= True)  